In [ ]:
import networkx as nx
import pandas as pd
from functions import *
import shutil


C:\Users\glaes\AppData\Local\Temp\ipykernel_18084\2927877141.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Load/Check Graph 
manually updates the graph into G recommended before running any cell   

selects a random hs ls and ns system and prints how the nodes currently look like

In [6]:
# Load graph
BACKUP_GRAPH_PATH = 'eve_universe_backup.graphml'
GRAPH_PATH = 'eve_universe.graphml'

G = nx.read_graphml(GRAPH_PATH)
print(f"Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.\n")
node_check(G)

Loaded graph with 8436 nodes and 6888 edges.

Loaded table with 8436 systems.

HS:
Node 30000079:
  name: Milu
  system_name: Milu
  security_status: 0.7696253061294556
  security_class: B
  constellation_id: 20000011
  star_id: 40004997.0
  planet_count: 11
  moon_count: 66
  station_count: 0
  stations: nan
  has_stargates: True
  region_id: 10000001
  region_name: Derelik
  constellation_name: Orshed
  faction_name: Ammatar Mandate
  industry_manufacturing: 0.0014
  industry_researching_time_efficiency: 0.0014
  industry_researching_material_efficiency: 0.0014
  industry_copying: 0.0014
  industry_invention: 0.0014
  industry_reaction: 0.0014
  Connected systems:
    - Tidacha (ID: 30000076)

LS:
Node 30001446:
  name: Otsasai
  system_name: Otsasai
  security_status: 0.3491613268852234
  security_class: C1
  constellation_id: 20000212
  star_id: 40092073.0
  planet_count: 8
  moon_count: 68
  station_count: 5
  stations: 60003604, 60003607, 60003811, ... (+2 more)
  has_stargates: 

### Commit to Backup Graph
Whenever the graph is in a not broken good state, you can push it to backup graph here. if you break the graph, you can restore it. 
(just copy and paste the contents of backup graph into the current one)

In [7]:

GRAPH_PATH = 'eve_universe.graphml'
BACKUP_GRAPH_PATH = 'eve_universe_backup.graphml'

# Copy the current graph to a backup location
shutil.copy(GRAPH_PATH, BACKUP_GRAPH_PATH)
print(f"Backup created at '{BACKUP_GRAPH_PATH}'.")

Backup created at 'eve_universe_backup.graphml'.


### Obtain/Update Info Tables
we create some info tables in util, so we can filter more efficiently or do looksup for more specific info

##### Systems Table

In [ ]:
OUTPUT_TABLE_PATH = "util/systems_table.csv"

# Build rows for the system table
rows = []
for node_id, data in G.nodes(data=True):
    try:
        sec = float(data.get("security_status", -1.0))
    except ValueError:
        sec = -1.0

    sec_type = classify_sec_type(sec)

    rows.append({
        "system_id": node_id,
        "name": data.get("system_name", data.get("name", "Unknown")),
        "constellation_id": data.get("constellation_id", "N/A"),
        "security_status": sec,
        "sec_type": sec_type
    })

# Convert to DataFrame
df_systems = pd.DataFrame(rows)
df_systems.to_csv(OUTPUT_TABLE_PATH, index=False)
print(f"Saved system info table to '{OUTPUT_TABLE_PATH}'.")

# Optional preview
df_systems.head()

Saved system info table to 'util/systems_table.csv'.


,system_id,name,constellation_id,security_status,sec_type
0,30000001,Tanoo,20000001,0.858324,HS
1,30000003,Akpivem,20000001,0.846292,HS
2,30000005,Sasta,20000001,0.814337,HS
3,30000007,Yuzier,20000001,0.906556,HS
4,30000002,Lashesih,20000001,0.751689,HS


##### Station Table

In [3]:
update_system_table(G)


Saved system metadata to 'util/systems_table.csv'.


##### small helper functions. that creates a station serice lookup file, if you want to see what could be available
station_services.json  
no need to rerun this in the near future. 

In [2]:
extract_station_services()


Saved 26 unique station services to 'util/station_services.json'.


##### Obtain/Update Region/Constallation Map
no need to rerun this in the near future

In [3]:
build_system_constellation_region_table(G)

Fetching constellations:   0%|          | 0/1174 [00:00<?, ?it/s]

Fetching regions:   0%|          | 0/112 [00:00<?, ?it/s]

Saved system/constellation/region table to 'util/system_constellation_region.csv'.


## Graph Refinement
the cells below are intended to modify or update the universe graph

##### Add/Update System Data

In [ ]:
load_system_data_to_graph(G)
nx.write_graphml(G, GRAPH_PATH)


Graph updated with system metadata from CSV.


##### Add/Update Region Data
based on the util/system_constellation_region.csv table

In [ ]:
update_graph_with_region_info(G)
nx.write_graphml(G, GRAPH_PATH)


##### Add/Update Faction Info based on region to faction map
I made that map myself, It is region and not faction specific, it might not be totally accurate in all cases

In [ ]:
assign_factions_by_region(G)
nx.write_graphml(G, GRAPH_PATH)


Updated 'faction_name' field for all systems.


### Update/Add Industry indeces. 

In [5]:
update_graph_with_industry_indices(G)
nx.write_graphml(G, GRAPH_PATH)


Fetching industry index data...


Updating graph with industry data:   0%|          | 0/5432 [00:00<?, ?it/s]

Updated industry indices for 5432 systems. Skipped 0 systems (not in graph).


### Update System activity routine
adds an entry to the system activity csv files with timestaps for the last 24 hours

In [5]:
update_activity_snapshots(G)

Updated activity data for 8436 systems.
